In [1]:
# Import modules
import numpy as np
from collections import Counter as Counter
import copy
import math
from operator import itemgetter as itemgetter

In [2]:
## Prep data
# Create probability counts using training data
# Create dictionary containers
Pwgt = dict()
Ptgt = dict()
# Read text file
with open('pos/pos_train.txt', 'r') as infile:
    # Loop through each line of text
    for line in infile:
        # Set initial "previous tag" to START if at initial word
        prev_tag = 'START'
        # Split line of text into "word-slash-strings" and loop
        for wst in line.strip().split(' '):
            # Extact word and tag
            wt = wst.split('/')
            tag = wt.pop()
            word = '/'.join(wt)
            # Count occurences of words and tags
            if tag not in Pwgt:
                Pwgt[tag] = dict()
            if word not in Pwgt[tag]:
                Pwgt[tag][word] = 0
            Pwgt[tag][word] += 1
            if prev_tag not in Ptgt:
                Ptgt[prev_tag] = dict()
            if tag not in Ptgt[prev_tag]:
                Ptgt[prev_tag][tag] = 0
            Ptgt[prev_tag][tag] += 1
            prev_tag = copy.deepcopy(tag)
        # Account for end of sentence
        tag = 'END'
        if prev_tag not in Ptgt:
            Ptgt[prev_tag] = dict()
        if tag not in Ptgt[prev_tag]:
            Ptgt[prev_tag][tag] = 0
        Ptgt[prev_tag][tag] += 1
# Determine most frequent tag
tags = Pwgt.keys()
n = 0;
for tag in tags:
    if sum(Pwgt[tag].values()) > n:
        mfreq_tag = tag
        n = sum(Pwgt[tag].values())

In [3]:
# Calculate log probabilities from counts
for tag in Pwgt:
    obs = sum(Pwgt[tag].values())
    for word in Pwgt[tag]:
        Pwgt[tag][word] = math.log(Pwgt[tag][word]/obs, 10)
for ptag in Ptgt:
    obs = sum(Ptgt[ptag].values())
    for ctag in Ptgt[ptag]:
        Ptgt[ptag][ctag] = math.log(Ptgt[ptag][ctag]/obs, 10)

In [4]:
# Test performance on test file
# Create variable "aliases"
ctags = list(Pwgt.keys())
ptags = list(Ptgt.keys())
mftag = [i for i, x in enumerate(ctags) if x == mfreq_tag][0]
# Create performance tracking variables
corr = 0
wrong = 0
# Read text file
k = 0
with open('pos/pos_test.txt', 'r') as infile:
    # Loop through each line of text
    for line in infile:
        # Reinitialize tracking variables for words, tags and paths
        word = []
        tag = []
        paths = [[]] * len(ctags)
        prev_paths = copy.deepcopy(paths)
        # Split line of text into "word-slash-strings"
        wst = line.strip().split(' ')
        # Loop through "word-slash-strings"
        for n in range(0, len(wst) + 1):
            # Set initial "previous tag" to START if at initial word
            if not n:
                prev_tags = ['START']
            else:
                prev_tags = copy.deepcopy(ctags)
                prev_paths = copy.deepcopy(paths)
                prev_probs_ct = copy.deepcopy(probs_ct)
            # Extact word and tag
            if n < len(wst):
                wt = wst[n].split('/')
                tag += [wt[-1]]
                wt.pop()
                word += ['/'.join(wt)]
            else:
                word += ['END']
                tag += ['END']
            # Calculate best path to reach the current tag if the current tag was that tag
            if n < len(wst):
                tloop = range(0, len(ctags))
            else:
                tloop = range(0, 1)
            probs_ct = []
            ptids = []
            for t in tloop:
                # Calculate probability of word given current tag
                if (word[n] not in Pwgt[ctags[t]]) and not (tag[n] is 'END'):
                    prob_wgt = np.log10(0)
                elif tag[n] is 'END':
                    prob_wgt = np.log10(1)
                else:
                    prob_wgt = Pwgt[ctags[t]][word[n]]
                # Loop through previous tags to find best path to current tag
                probs = []
                for pt in range(0, len(prev_tags)):
                    # Calculate probability of current tag given previous tag
                    if not (tag[n] is 'END'):
                        ctag = ctags[t]
                    else:
                        ctag = tag[n]
                    if ctag not in Ptgt[prev_tags[pt]]:
                        prob_tgt = np.log10(0)
                    else:
                        prob_tgt = Ptgt[prev_tags[pt]][ctag]
                    # Extract probability of previous tag
                    if prev_tags == ['START']:
                        prob_pt = np.log10(1)
                    else:
                        prob_pt = prev_probs_ct[pt]
                    # Add probabilities to get probability of current tag
                    probs += [prob_wgt + prob_tgt + prob_pt]
                # Take max over previous tags to find best path to current tag
                ptid, prob_ct = max(enumerate(probs), key = itemgetter(1))
                ptids += [ptid]
                probs_ct += [prob_ct]
                if probs_ct[t] == -np.inf:
                    ptids[t] = np.nan
            paths = []
            for t in tloop:
                if not np.isnan(ptids[t]):
                    paths += [prev_paths[ptids[t]] + [prev_tags[ptids[t]]]]
                else:
                    paths += [np.nan]
            # Account for case that one of the probilities have not been observed by setting current tag to most frequent
            # tag
            if sum(np.isnan(ptids)) == len(ptids):
                # Account for initial boundary condition
                if not n:
                    paths[mftag] = [prev_tags[0]]
                    probs_ct[mftag] = np.log10(1)
                else:
                    ind = np.where(np.logical_and(np.equal(prev_probs_ct, max(prev_probs_ct)),
                                                  [isinstance(l, list) for l in prev_paths]))
                    ind = ind[0][0]
                    paths[mftag] = prev_paths[ind] + [prev_tags[ind]]
                    probs_ct[mftag] = np.log10(1)
        # Extract best path
        best_path = paths[0]
        # Remove "START" tag
        best_path.pop(0)
        # Iterate performance
        for n in range(0, len(tag) - 1):
            if best_path[n] == tag[n]:
                corr += 1
            else:
                wrong += 1
        k += 1
        print(k)
# Display performance
print('Percent Correct: {:.2f}\nPercent Wrong: {:.2f}\n'.format(corr/(corr + wrong), wrong/(corr + wrong)))

/home/test/.local/lib/python3.5/site-packages/ipykernel_launcher.py:49: RuntimeWarning: divide by zero encountered in log10
/home/test/.local/lib/python3.5/site-packages/ipykernel_launcher.py:63: RuntimeWarning: divide by zero encountered in log10


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
